# Fine Tuned Model for Base Data Analysis

**TLDR: Fine tuned an OpenAI model to write SQL queries to analyze Base blockchain data.**

[Base](https://base.org/) blockchain data provided by [Flipside](https://flipsidecrypto.xyz/).

Queries were sourced from Base/Flipside dashboards by [@adriaparcerisas](https://flipsidecrypto.xyz/adriaparcerisas/base-active-users-fBkhsx), [@AliTslm](https://flipsidecrypto.xyz/alitaslimi/base-mainnet-base-mainnet-s0oITj), [@jackguy](https://flipsidecrypto.xyz/jackguy/base-onchain-summer-nft-dashboard-base-onchain-summer-nft-dashboard-VfYxS8), [@piper](https://flipsidecrypto.xyz/piper/base-onchain-summer-starts-08-09-23-base---onchain-summer-starts-08.09.23-pI8o4d), and [@saeedmzn](https://flipsidecrypto.xyz/saeedmzn/base-rank-check-base-rank-check-6DQsjX).



In [ ]:
%pip install openai

In [ ]:
import os
import openai
import pandas as pd
import csv
import json
import getpass

openai.api_key = getpass.getpass()

··········


In [ ]:
# CSV file with question/query examples
queries_df = pd.read_csv("Flipside Base Queries.csv")
queries_df.head()

,system,user,assistant
0,You are an SQL assistant. You write SQL to que...,what is the number of daily active users trans...,WITH\n daus as (\n SELECT\n distinct from_...
1,You are an SQL assistant. You write SQL to que...,what is the number of daily active users swapp...,WITH\n daus as (\n SELECT\n distinct from_...
2,You are an SQL assistant. You write SQL to que...,what is the number of daily active users using...,WITH\n daus as (\n SELECT\n distinct nft_fr...
3,You are an SQL assistant. You write SQL to que...,what is the number of daily Base transactions ...,WITH\n daus as (\n SELECT\n distinct from_...
4,You are an SQL assistant. You write SQL to que...,what is the daily and total number of transact...,"select\r\n block_timestamp::date as ""Date"",\r..."


In [ ]:
# Convert CSV file to proper JSON format to fine tune
with open('Flipside Base Queries.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    json_messages_list = []
    for row in csv_reader:
        json_messages = [
            {"role": "system", "content": row["system"]},
            {"role": "user", "content": row["user"]},
            {"role": "assistant", "content": row["assistant"]}
        ]
        json_messages_list.append({"messages": json_messages})

    # Write to JSON file
    with open('queries.json', 'w') as json_file:
        for json_message in json_messages_list:
            json.dump(json_message, json_file)
            json_file.write('\n')


In [ ]:
# Create file with examples to fine tune model
openai.File.create(
  file=open("queries.json", "rb"),
  purpose='fine-tune'
)

<File file id=file-Hyb3bEqTEJxuKu0sf2uz48Il at 0x7e951e576e80> JSON: {
  "object": "file",
  "id": "file-Hyb3bEqTEJxuKu0sf2uz48Il",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 73424,
  "created_at": 1692765793,
  "status": "uploaded",
  "status_details": null
}

In [ ]:
# Start fine tuning Job (Took < 10 mins)
openai.FineTuningJob.create(training_file="file-Hyb3bEqTEJxuKu0sf2uz48Il", model="gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-ajDSfTME882gihBIwNpL23pq at 0x7e951e5a63e0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-ajDSfTME882gihBIwNpL23pq",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692765823,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-62MyjxLwbbsMgywL5uJQVgjM",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-Hyb3bEqTEJxuKu0sf2uz48Il",
  "hyperparameters": {
    "n_epochs": 5
  },
  "trained_tokens": null
}

In [ ]:
# Find the name of the fine tuning job
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x7e951e705e40> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-ajDSfTME882gihBIwNpL23pq",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1692765823,
      "finished_at": 1692766383,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7qa9UP0a",
      "organization_id": "org-62MyjxLwbbsMgywL5uJQVgjM",
      "result_files": [
        "file-CYxIMhNm4hJq8otgcb9Nb04N"
      ],
      "status": "succeeded",
      "validation_file": null,
      "training_file": "file-Hyb3bEqTEJxuKu0sf2uz48Il",
      "hyperparameters": {
        "n_epochs": 5
      },
      "trained_tokens": 79975
    }
  ],
  "has_more": false
}

In [ ]:
# Check the state of the fine tuning. Once finished "fine_tuned_model" will have a name.
openai.FineTuningJob.retrieve("ftjob-ajDSfTME882gihBIwNpL23pq")

<FineTuningJob fine_tuning.job id=ftjob-ajDSfTME882gihBIwNpL23pq at 0x7e9516659350> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-ajDSfTME882gihBIwNpL23pq",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692765823,
  "finished_at": 1692766383,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7qa9UP0a",
  "organization_id": "org-62MyjxLwbbsMgywL5uJQVgjM",
  "result_files": [
    "file-CYxIMhNm4hJq8otgcb9Nb04N"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-Hyb3bEqTEJxuKu0sf2uz48Il",
  "hyperparameters": {
    "n_epochs": 5
  },
  "trained_tokens": 79975
}

In [ ]:
def base_flipside_model(prompt):

  system_description = """
  "You are an SQL assistant. You write SQL to query Flipside's Snowflake database with data for the Base blockchain.

  The database has the following tables and columns:
  [{""TABLE_NAME"":""dim_contracts"",""COLUMN_NAMES"":[""created_block_number"",""decimals"",""creator_address"",""address"",""created_tx_hash"",""symbol"",""name"",""created_block_timestamp""]},{""TABLE_NAME"":""dim_labels"",""COLUMN_NAMES"":[""address_name"",""label_subtype"",""label_type"",""address"",""blockchain"",""creator"",""project_name""]},{""TABLE_NAME"":""ez_decoded_event_logs"",""COLUMN_NAMES"":[""origin_to_address"",""origin_function_signature"",""event_removed"",""tx_hash"",""block_number"",""full_decoded_log"",""tx_status"",""contract_name"",""data"",""event_index"",""event_name"",""block_timestamp"",""origin_from_address"",""contract_address"",""decoded_log"",""topics""]},{""TABLE_NAME"":""ez_nft_transfers"",""COLUMN_NAMES"":[""event_type"",""tx_hash"",""project_name"",""block_number"",""nft_from_address"",""event_index"",""nft_to_address"",""nft_address"",""erc1155_value"",""tokenid"",""block_timestamp""]},{""TABLE_NAME"":""fact_blocks"",""COLUMN_NAMES"":[""blockchain"",""size"",""parent_hash"",""gas_limit"",""sha3_uncles"",""block_header_json"",""block_number"",""network"",""uncle_blocks"",""extra_data"",""block_timestamp"",""difficulty"",""receipts_root"",""total_difficulty"",""hash"",""gas_used"",""tx_count""]},{""TABLE_NAME"":""fact_decoded_event_logs"",""COLUMN_NAMES"":[""event_index"",""block_timestamp"",""decoded_log"",""tx_hash"",""event_name"",""contract_address"",""full_decoded_log"",""block_number""]},{""TABLE_NAME"":""fact_event_logs"",""COLUMN_NAMES"":[""tx_status"",""data"",""tx_hash"",""_log_id"",""event_index"",""block_timestamp"",""topics"",""block_number"",""event_removed"",""contract_address"",""origin_from_address"",""origin_to_address"",""origin_function_signature""]},{""TABLE_NAME"":""fact_hourly_token_prices"",""COLUMN_NAMES"":[""is_imputed"",""decimals"",""symbol"",""token_address"",""hour"",""price""]},{""TABLE_NAME"":""fact_token_transfers"",""COLUMN_NAMES"":[""to_address"",""block_timestamp"",""from_address"",""raw_amount"",""raw_amount_precise"",""origin_function_signature"",""origin_to_address"",""contract_address"",""_log_id"",""tx_hash"",""origin_from_address"",""block_number""]},{""TABLE_NAME"":""fact_traces"",""COLUMN_NAMES"":[""type"",""tx_hash"",""to_address"",""trace_index"",""block_number"",""gas"",""from_address"",""eth_value"",""eth_value_precise_raw"",""block_timestamp"",""data"",""trace_status"",""error_reason"",""eth_value_precise"",""input"",""sub_traces"",""identifier"",""output"",""gas_used"",""tx_status""]},{""TABLE_NAME"":""fact_transactions"",""COLUMN_NAMES"":[""cumulative_gas_used"",""tx_fee_precise"",""tx_fee"",""l1_gas_used"",""position"",""l1_fee"",""eth_value"",""input_data"",""block_hash"",""l1_gas_price"",""l1_fee_precise"",""v"",""gas_price"",""status"",""effective_gas_price"",""block_timestamp"",""l1_submission_details"",""nonce"",""eth_value_precise_raw"",""tx_hash"",""from_address"",""max_priority_fee_per_gas"",""eth_value_precise"",""gas_limit"",""block_number"",""origin_function_signature"",""l1_fee_scalar"",""r"",""gas_used"",""s"",""max_fee_per_gas"",""to_address""]}]

  Please respond to a user's question about the Base blockchain with a SQL query."
  """

  completion = openai.ChatCompletion.create(
    model="ft:gpt-3.5-turbo-0613:personal::7qa9UP0a",
    messages=[
      {"role": "system", "content": system_description},
      {"role": "user", "content": prompt}
    ]
  )

  return completion.choices[0].message

In [ ]:
prompt = "What is the daily number of transactions on Base?"
base_flipside_model(prompt)

<OpenAIObject at 0x7e95166388b0> JSON: {
  "role": "assistant",
  "content": "SELECT\n  block_timestamp,\n  COUNT(*) AS \"Number of Transactions\"\nFROM\n  base.core.fact_transactions\nGROUP BY\n  block_timestamp\nORDER BY\n  block_timestamp\u1ee7a"
}